In [33]:
from utils import process_text


In [34]:
import os 
os.chdir('..')
print(os.getcwd())
print(os.listdir())

/home/matcha/new_matcha
['.ipynb_checkpoints', 'Matcha-TTS', 'eval_audio']


In [35]:
import torch
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import soundfile as sf
import sys 

In [36]:
sys.path.append('Matcha_TTS_main/') # add the Matcha_TTS_main (code from the original repo) directory to the path for the vecoder


In [37]:
from main import MatchaTTS 
# for Hifigan
from matcha.hifigan.config import v1
from matcha.hifigan.denoiser import Denoiser
from matcha.hifigan.env import AttrDict
from matcha.hifigan.models import Generator as HiFiGAN

In [67]:
# load the model checkpoints
# matcha_checkpoint_path = "logs/train/ljspeech/runs/2026-01-09_12-40-08/checkpoints/checkpoint_epoch=100.ckpt"
hifigan_checkpoint_path = "/home/matcha/new_matcha/Matcha-TTS/checkpoints/generator_v1"
matcha_checkpoint_path = "/home/matcha/new_matcha/Matcha-TTS/logs/train/ljspeech/runs/2026-01-09_12-40-08/checkpoints/checkpoint_epoch=190.ckpt"

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [65]:
def load_model(checkpoint_path):
    model = MatchaTTS.load_from_checkpoint(
        checkpoint_path, 
        map_location=device,
        weights_only=False 
    )
    model.eval()
    return model

model = load_model(matcha_checkpoint_path)
print(f"Model loaded!")

Model loaded!


In [69]:
def load_vocoder(checkpoint_path):
    h = AttrDict(v1)
    hifigan = HiFiGAN(h).to(device)
    hifigan.load_state_dict(torch.load(checkpoint_path, map_location=device)['generator'])
    _ = hifigan.eval()
    hifigan.remove_weight_norm()
    return hifigan

vocoder = load_vocoder(hifigan_checkpoint_path)
denoiser = Denoiser(vocoder, mode='zeros')

Removing weight norm...


/tmp/ipykernel_63950/772254795.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hifigan.load_state_dict(torch.load(checkpoint_path, map_location=device)['generator'])


In [70]:
test_text ="thank you for listening and welcome to the future of speech. Matcha TTS"
result = process_text(test_text, device)


Processing complete!


In [71]:
## Number of ODE Solver steps
n_timesteps = 10

## Changes to the speaking rate
length_scale=1.0

## Sampling temperature
temperature = 0.667

In [72]:
# Generate mel-spectrogram using the model
start_time = dt.datetime.now()

with torch.inference_mode():  # Disable gradients for inference
    output = model.synthesise(
        result['x'],              # Phoneme IDs tensor
        result['x_lengths'],      # Length of sequence
        n_timesteps=n_timesteps,
        temperature=temperature,
        length_scale=length_scale
    )

end_time = dt.datetime.now()
synthesis_time = (end_time - start_time).total_seconds()

print(f" Mel-spectrogram generated in {synthesis_time:.2f} seconds!")


 Mel-spectrogram generated in 0.15 seconds!


In [73]:
# Extract the mel-spectrogram
mel=output['mel']
mel_to_draw = mel.squeeze().cpu().numpy()  #

In [3]:
with torch.inference_mode(): # Wrap vocoder and denoiser in inference mode
    audio = vocoder(mel).clamp(-1, 1)
    audio = denoiser(audio.squeeze(0), strength=0.00025).cpu().squeeze()
audio.cpu().squeeze()

# ipd.display(ipd.Audio(audio, rate=22050))
#save the audio to a wav file
# sf.write('output/generated_audio_synthes_example_original_model.wav', audio.numpy(), 22050)